## Quick introduction to generating counterfactual explanations using DiCE

In [ ]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

DiCE requires two inputs: a training dataset and a pre-trained ML model. It can also work without access to the full dataset (see this [notebook](DiCE_with_private_data.ipynb) for advanced examples).

### Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in **dice_ml.utils.helpers** module. 

In [ ]:
dataset = helpers.load_adult_income_dataset()

This dataset has 8 features. The outcome is income which is binarized to 0 (low-income, <=50K) or 1 (high-income, >50K). 

In [ ]:
dataset.head()

In [ ]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

Given this dataset, we construct a data object for DiCE. Since continuous and discrete features have different ways of perturbation, we need to specify the names of the continuous features. DiCE also requires the name of the output variable that the ML model will predict.

In [ ]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### Loading the ML model

Below, we use a pre-trained ML model which produces high accuracy comparable to other baselines. For convenience, we include the sample trained model with the DiCE package.

In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable

import numpy as np

from dice_ml.model_interfaces.pytorch_model import PyTorchModel

In [ ]:
inp_shape= len(d.encoded_feature_names)

ML_modelpath = helpers.get_adult_income_modelpath()
m = PyTorchModel(inp_shape)

learning_rate = 0.001
# Default Batch Size of Keras
batch_size = 32
optimizer = optim.Adam([
    {'params': filter(lambda p: p.requires_grad, m.ann_model.parameters()) }
], lr=learning_rate)
crieterion= nn.CrossEntropyLoss()

In [ ]:
#Pre Trained
base_model_dir= '../dice_ml/utils/sample_trained_models/'
dataset_name= 'adult'
path=base_model_dir+dataset_name+'-pytorch.pth'
m.load_state_dict(torch.load(path))
m.eval()

In [ ]:
# Dataset for training Black Box Model
train_data_vae= d.data_df.copy()

#Creating list of encoded categorical and continuous feature indices
encoded_categorical_feature_indexes = d.get_data_params()[2]     
encoded_continuous_feature_indexes=[]
data_size= len(d.encoded_feature_names)
for i in range(data_size):
    valid=1
    for v in encoded_categorical_feature_indexes:
        if i in v:
            valid=0
    if valid:
        encoded_continuous_feature_indexes.append(i)            
encoded_start_cat = len(encoded_continuous_feature_indexes)
        
#One Hot Encoding for categorical features
encoded_data = d.one_hot_encode_data(train_data_vae)

# The output/outcome variable position altered due to one_hot_encoding for categorical features: (Cont feat, Outcome, Cat feat) 
# Need to rearrange columns such that outcome variable comes at the last
cols = list(encoded_data.columns)
cols = cols[:encoded_start_cat] + cols[encoded_start_cat+1:] + [cols[encoded_start_cat]]
encoded_data = encoded_data[cols]     

#Normlization for conitnuous features
encoded_data= d.normalize_data(encoded_data)
print(encoded_data.columns)
dataset = encoded_data.to_numpy()

#Train, Val, Test Splits
np.random.shuffle(dataset)
test_size= int(0.2*dataset.shape[0])
val_dataset= dataset[:test_size]
train_dataset= dataset[test_size:]

In [ ]:
#Training
for epoch in range(50):
    np.random.shuffle(train_dataset)
    train_batches= np.array_split( train_dataset, train_dataset.shape[0]//batch_size ,axis=0 )    
    print('Epoch: ', epoch)
    train_acc=0.0
    for i in range(len(train_batches)):    
        optimizer.zero_grad()
        train_x= torch.tensor( train_batches[i][:,:-1] ).float() 
        train_y= torch.tensor( train_batches[i][:,-1], dtype=torch.int64 )
        
        out= m(train_x)
        train_acc += torch.sum( torch.argmax(out, axis=1) == train_y )
        
        # Cross Entropy Loss
        loss= crieterion(out, train_y)
        #L2 Regularization
        weight_norm = torch.tensor(0.)
        for w in m.ann_model.parameters():
            weight_norm += w.norm().pow(1)
        loss+= 0.001*weight_norm
        
        loss.backward()
        optimizer.step()
    print(train_acc, len(train_dataset))     

In [ ]:
# Validation        
np.random.shuffle(val_dataset)
train_batches= np.array_split( val_dataset, val_dataset.shape[0]//batch_size ,axis=0 )    
val_acc=0.0
for i in range(len(train_batches)):    
    optimizer.zero_grad()
    train_x= torch.tensor( train_batches[i][:,:-1] ).float() 
    train_y= torch.tensor( train_batches[i][:,-1], dtype=torch.int64 )
    out= m(train_x)
    val_acc += torch.sum( torch.argmax(out, axis=1) == train_y )
print(val_acc, len(val_dataset))	

#Saving the Black Box Model
base_model_dir= '../dice_ml/utils/sample_trained_models/'
dataset_name= 'adult'
path=base_model_dir+dataset_name+'-pytorch.pth'
torch.save(m.state_dict(), path)   

For an example of how to train your own model, check out [this](DiCE_with_advanced_options.ipynb) notebook. 

### Generate feasible counterfactuals

Based on the data object *d* and the model object *m*, we can now instantiate the DiCE class for generating explanations. 

In [ ]:
from dice_ml.dice_interfaces.dice_base_gencf import DiceBaseGenCF

In [ ]:
# initiate DiCE
exp = DiceBaseGenCF(d, m)
exp.train(pre_trained=0)

DiCE is a form of a local explanation and requires an query input whose outcome needs to be explained. Below we provide a sample input whose outcome is 0 (low-income) as per the ML model object *m*. 

In [ ]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

Given the query input, we can now generate counterfactual explanations to show perturbed inputs from the original input where the ML model outputs class 1 (high-income). 

In [ ]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=15, desired_class="opposite")

In [ ]:
# visualize the results
dice_exp.visualize_as_dataframe()

That's it! You can try generating counterfactual explanations for other examples using the same code. You can also try out a use-case for sensitive data in [this](DiCE_with_private_data.ipynb) notebook. 

If you are curious about changing the number of explanations showns, feasibility of the explanations, or how to weigh different features for perturbing, check out how to change DiCE's behavior in this [advanced notebook](DiCE_with_advanced_options.ipynb).  

The counterfactuals generated above are slightly different from those shown in [our paper](https://arxiv.org/pdf/1905.07697.pdf), where the loss convergence condition was made more conservative for rigorous experimentation. To replicate the results in the paper, add an argument *loss_converge_maxiter=2* (the default value is 1) in the *exp.generate_counterfactuals()* method above. For more info, see *generate_counterfactuals()* method in [dice_ml.dice_interfaces.dice_tensorflow.py](../dice_ml/dice_interfaces/dice_tensorflow.py).

### ModelApprox 

In [ ]:
from dice_ml.dice_interfaces.dice_model_approx_gencf import DiceModelApproxGenCF

In [ ]:
# initiate DiCE
exp = DiceModelApproxGenCF(d, m)
exp.train(1, [[0]], 1, 10, pre_trained=1)

In [ ]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=15, desired_class="opposite")
# visualize the results
dice_exp.visualize_as_dataframe()